In [2]:
!python -V

Python 3.9.23


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pickle
import sklearn
import numpy as np

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-eperiment")



2025/12/12 02:51:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/12 02:51:19 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/Mlops_Zoom_camp/02-Experiment Tracking/mlruns/1', creation_time=1765408879095, experiment_id='1', last_update_time=1765408879095, lifecycle_stage='active', name='nyc-taxi-eperiment', tags={}>

In [3]:
def data_read_pre_processing(file_path):
    
   
    #reading the data from the defined path
    df = pd.read_parquet(file_path)
    
    #Converting the columns to timestamp
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    #Calculating duration of the trip
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
    
    #filetring the data 
    df = df[(df.duration>=1) & (df.duration<=60)]
    
    return df

In [4]:
df_train = data_read_pre_processing('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

df_val = data_read_pre_processing('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [5]:
import sys, sklearn
print(sys.executable)
print(sklearn.__version__)


/home/codespace/anaconda3/envs/exp-tracking-env/bin/python
1.3.0


In [6]:
#### Created new Feature
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO']  = df_val['PULocationID'].astype(str)  + '_' + df_val['DOLocationID'].astype(str)


In [7]:
### defining the train set 
categorical = ['PU_DO']#['PULocationID','DOLocationID']
numerical = ['trip_distance']

### Converting categorical variable into string for preprocessing
df_train[categorical] = df_train[categorical].astype(str)
train_dicts = df_train[categorical+numerical].to_dict(orient = 'records') 

### Converting categorical variable into string for preprocessing
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical+numerical].to_dict(orient = 'records') 

In [8]:
### Vectoriser
dv = DictVectorizer()

In [9]:
## vectorizing traing Features
X_train = dv.fit_transform(train_dicts)
X_train

<73908x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 147816 stored elements in Compressed Sparse Row format>

In [10]:
## vectorizing validation Features
X_val = dv.transform(val_dicts)
X_val

<61921x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 118585 stored elements in Compressed Sparse Row format>

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [13]:
# prediting the values for train set
y_predict_train = lr.predict(X_train)

# prediting the values for train set
y_predict_val = lr.predict(X_val)

In [14]:
import inspect
inspect.signature(mean_squared_error)


<Signature (y_true, y_pred, *, sample_weight=None, multioutput='uniform_average', squared=True)>

In [15]:
rmse = np.sqrt(mean_squared_error(y_train,y_predict_train))
rmse

5.699564118198979

In [16]:
rmse_val =  mean_squared_error(y_val,y_predict_val,squared = False)
rmse_val

7.758715209663881

In [17]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr),f_out)

# Checking a different Model like LASSO or Ridge

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer","Tej")

    mlflow.log_param("train-data-url","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-url","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    
    alpha =0.1
    mlflow.log_param("alpha",alpha)
    lr_new = Lasso(alpha)
    lr_new.fit(X_train,y_train)
    
    y_pred_new = lr_new.predict(X_val)
    rmse = mean_squared_error(y_val,y_pred_new,squared = False)
    mlflow.log_metric("rmse",rmse)
    mlflow.log_artifact(local_path = "models/lin_reg.bin", artifact_path = "models_pickle")

## Performing Hyper Parameter Tuning and loggoing the best results in ML flow  

In [20]:
import xgboost as xgb

from hyperopt import fmin,hp,tpe,STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [21]:
train = xgb.DMatrix(X_train,label = y_train)
valid = xgb.DMatrix(X_val,label = y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params =params,
            dtrain =train,
            num_boost_round =1000,
            evals=[(valid,"validation")],
            early_stopping_rounds = 50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val,y_pred,squared = False)
        mlflow.log_metric("rmse", rmse)

    return {"loss":rmse,'status':STATUS_OK}
        

        
        

In [24]:
search_space = {
    'max_depth' :scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate' :hp.loguniform('learning_rate',-3,0),
    'reg_alpha' :hp.loguniform('reg_alpha',-5,-1),
    'reg_lambda' :hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight' :hp.loguniform('min_child_weight',-1,3),
    'objective' :'reg:squarederror',
    'seed' : 42,
}

best_result =fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
    )

[0]	validation-rmse:10.90308                                                                                                                                                                              
[1]	validation-rmse:9.87851                                                                                                                                                                               
[2]	validation-rmse:9.08981                                                                                                                                                                               
[3]	validation-rmse:8.48934                                                                                                                                                                               
[4]	validation-rmse:8.03668                                                                                                                                                                 

KeyboardInterrupt: 

In [22]:
import os, pickle
with mlflow.start_run():

    best_params = {
    "learning_rate" : 0.1832607471256377,
    "max_depth" : 53,
    "min_child_weight" : 1.5248259426208242,
    "objective" : "reg:squarederror",
    "reg_alpha" : 0.03914337949093102,
    "reg_lambda" : 0.20458604590859147,
    "seed" : 42,
    }
    mlflow.log_params(best_params)
    booster = xgb.train(
            params =best_params,
            dtrain =train,
            num_boost_round =100,
            evals=[(valid,"validation")],
            early_stopping_rounds = 50
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val,y_pred,squared = False)
    mlflow.log_metric("rmse", rmse)
    os.makedirs("models", exist_ok=True)
    with open('models/preprocessor.b','wb') as f_out:
       pickle.dump(dv,f_out)
    mlflow.log_artifact("models/preprocessor.b",artifact_path = "preprocessor")
    
    mlflow.xgboost.log_model(booster,name="xgb_model")
    

[0]	validation-rmse:10.74801
[1]	validation-rmse:9.63418
[2]	validation-rmse:8.79797
[3]	validation-rmse:8.17701
[4]	validation-rmse:7.72271
[5]	validation-rmse:7.39379
[6]	validation-rmse:7.15383
[7]	validation-rmse:6.98266
[8]	validation-rmse:6.85603
[9]	validation-rmse:6.76075
[10]	validation-rmse:6.69187
[11]	validation-rmse:6.63994
[12]	validation-rmse:6.60079
[13]	validation-rmse:6.57000
[14]	validation-rmse:6.54547
[15]	validation-rmse:6.52617
[16]	validation-rmse:6.51044
[17]	validation-rmse:6.49776
[18]	validation-rmse:6.48631
[19]	validation-rmse:6.47826
[20]	validation-rmse:6.47111
[21]	validation-rmse:6.46375
[22]	validation-rmse:6.45912
[23]	validation-rmse:6.45605
[24]	validation-rmse:6.45350
[25]	validation-rmse:6.45021
[26]	validation-rmse:6.44861
[27]	validation-rmse:6.44696
[28]	validation-rmse:6.44587
[29]	validation-rmse:6.44487
[30]	validation-rmse:6.44381
[31]	validation-rmse:6.44071
[32]	validation-rmse:6.43863
[33]	validation-rmse:6.43708
[34]	validation-rmse:6.

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [02:53:13] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/12/12 02:53:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [28]:
with mlflow.start_run():

    best_params = {
        "learning_rate" : 0.1832607471256377,
        "max_depth" : 53,
        "min_child_weight" : 1.5248259426208242,
        "objective" : "reg:squarederror",
        "reg_alpha" : 0.03914337949093102,
        "reg_lambda" : 0.20458604590859147,
        "seed" : 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # Log preprocessor inside artifacts
    with open("preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor")
    
    result = mlflow.xgboost.log_model(
    xgb_model=booster,
    artifact_path="model",
    registered_model_name="nyc-taxi-regressor"
    )

    print("Model logged at:")
    print(result.model_uri)


[0]	validation-rmse:10.74801
[1]	validation-rmse:9.63418
[2]	validation-rmse:8.79797
[3]	validation-rmse:8.17701
[4]	validation-rmse:7.72271
[5]	validation-rmse:7.39379
[6]	validation-rmse:7.15383
[7]	validation-rmse:6.98266
[8]	validation-rmse:6.85603
[9]	validation-rmse:6.76075
[10]	validation-rmse:6.69187
[11]	validation-rmse:6.63994
[12]	validation-rmse:6.60079
[13]	validation-rmse:6.57000
[14]	validation-rmse:6.54547
[15]	validation-rmse:6.52617
[16]	validation-rmse:6.51044
[17]	validation-rmse:6.49776
[18]	validation-rmse:6.48631
[19]	validation-rmse:6.47826
[20]	validation-rmse:6.47111
[21]	validation-rmse:6.46375
[22]	validation-rmse:6.45912
[23]	validation-rmse:6.45605
[24]	validation-rmse:6.45350
[25]	validation-rmse:6.45021
[26]	validation-rmse:6.44861
[27]	validation-rmse:6.44696
[28]	validation-rmse:6.44587
[29]	validation-rmse:6.44487
[30]	validation-rmse:6.44381
[31]	validation-rmse:6.44071
[32]	validation-rmse:6.43863
[33]	validation-rmse:6.43708
[34]	validation-rmse:6.

2025/12/12 03:10:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:10:26] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/12/12 03:10:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged at:
models:/m-600597f14d494833bfc6190b02c02794


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '7' of model 'nyc-taxi-regressor'.


In [25]:
result = mlflow.xgboost.log_model(
    xgb_model=booster,
    artifact_path="model",
    registered_model_name="nyc-taxi-regressor"
)

print("Model logged at:")
print(result.model_uri)


2025/12/12 03:07:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:07:32] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/12/12 03:07:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged at:
models:/m-147b454879804007b41e2b03166b0478


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-regressor'.
